In [1]:
import base64
import requests
# urlencode - wrap this around a data dictionary to create a URL ready string 
from urllib.parse import urlencode

import datetime
from secrets import client_id, client_secret


In [2]:
def convert_ms(millis):
    '''
    convert milliseconds into seconds
    '''
    millis = int(millis)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)
#     hours=(millis/(1000*60*60))%24
    min2sec = minutes * 60 
    
    return min2sec + seconds
    
#     return ("%d:%d:%d" % (hours, minutes, seconds))

def parse_track(item):
    '''
    Parses one item from the playlist - return artist dictionary
    '''
    track = item['track']
    
    track_name = item['track']['name']
    track_artists = [artist['name'] for artist in track['artists']]
    lead, feature = track_artists[0], track_artists[1:] 
    
    
    album_name, album_image = track['album']['name'], track['album']['images'][0]['url']
    release_date = track['album']['release_date']
    album_popularity = track['popularity']
    duration = convert_ms(track['duration_ms'])
    
    return dict(lead = {'track': track_name,
                                'album': album_name, 
                                'albumImage': album_image,
                                'release_date': release_date,
                                'popularity': album_popularity, 
                                'duration': duration})

In [3]:
class SpotifyAPI():
    # Placeholder for attributes    
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret= None
    token_endpoint = 'https://accounts.spotify.com/api/token'
    user_id = '12185809439'
    playlist_endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
    
    
    def __init__(self, api_key, client_id=None, client_secret=None, *args, **kwargs):
        # super init - allows it to call any class that its inheriting from        
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.api_key = api_key
    
    def get_client_credentials(self):
        '''
        Returns a base64 encoded string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("Must set client_id and client_secret")
        client_cred = f"{client_id}:{client_secret}"
        client_cred_b64 = base64.b64encode(client_cred.encode())
        return client_cred_b64.decode()
    
    def get_token_headers(self):
        client_cred_b64 = self.get_client_credentials()
        return {
        "Authorization": f"Basic {client_cred_b64}"
    }
    
    def get_bearer_headers(self):
        api_key = self.api_key
        return {
        "Authorization": f"Bearer {api_key}"
    }
    
    
    def get_token_data(self):
        return {
        "grant_type": "client_credentials"
    }
    
    def auth(self):
        token_endpoint = self.token_endpoint
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_endpoint, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            return False
        
        print(f"Status Code: {r.status_code}")
        now = datetime.datetime.now()
        token_response = r.json()
        access_token = token_response['access_token']
        expires_in = token_response['expires_in'] # in seconds
        expires = now + datetime.timedelta(seconds=expires_in) # use datetime to calculate the time of expiration
        
        self.access_token = access_token 
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        self.token_res = token_response
        return True
    
    def get_my_playlists(self, limit, offset):
        user_id = '12185809439'
        playlist_endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
        headers = self.get_bearer_headers()
        data = urlencode({"limit": limit, 'offset':offset})
        # A URL ready string to get user playlists
        lookup_url = f"{playlist_endpoint}?{data}"
        
        res = requests.get(lookup_url, headers=headers)
        
        if res.status_code not in range(200, 299):
            raise Exception(res.json())
        
        self.my_playlists = res.json()['items']
        
    
    def get_playlist(self, playlist_id):
        '''
        Takes a playlistID and return the playlist
        '''
        headers = self.get_bearer_headers()
        playlist_endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
        
        res = requests.get(playlist_endpoint, headers=headers)
        
        if res.status_code not in range(200, 299):
            raise Exception(res.json())
        return res.json()

## Request an Auth Token here
token expires in an hour, find a way to refresh this token automatically in the future
https://developer.spotify.com/console/get-playlists/?user_id=12185809439&limit=10&offset=5

In [4]:
api_key = input('Enter Spotify Auth Token: ')
spotify = SpotifyAPI(api_key)

Enter Spotify Auth Token: BQAAujUbt-QITJO12uoV9qFazkLv_-ywvCoCq1mxSK1xVWtEU8oB8Llz6nP-y5z74QpkTwGrkaGOV-Yx3n1VR_37cvypvIQT4mMRVjH_UdwWEQXu0Xf3IOmZiuDREerNUG8gbTidYm1GTFAMYGl2ZjCBkHTSG1cr2YptJL_ITIYX1ImJ6yrAj3dL


In [112]:
spotify.get_my_playlists(limit=15, offset=5)
playlists_name = [playlist['name'] for playlist in spotify.my_playlists]
playlists_name

['My Shazam Tracks',
 '💣💥',
 'Sum Good Shit',
 '浮誇',
 'Memeingful',
 'activate your mojo',
 'goodbye aurabell 9/18-6/19',
 'OG',
 '廣東歌',
 'C',
 'Generic Trap',
 'K',
 'Tokyo Ghoul「東京喰種」',
 'whitegirl playlist',
 'Shazam Tracks']

## Find tracks in a playlists
Provide PlaylistID to retrieve contents of this playlist - https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlist/

In [7]:
top_2019_id = '37i9dQZF1EtsKZinWyvvzX'
top_2018_id = '37i9dQZF1EjefPCsnd'
top_2017_id = '37i9dQZF1E9PDuAPTNqRWN'
top_2016_id = '37i9dQZF1Cz1NXVJwDDfcU'

top_2019 = spotify.get_playlist("37i9dQZF1EtsKZinWyvvzX")
# Extract the items using top_20XX['items'] 

top_2019.keys()
top_2019['items']

[{'added_at': '2019-12-12T21:16:59Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
   'href': 'https://api.spotify.com/v1/users/',
   'id': '',
   'type': 'user',
   'uri': 'spotify:user:'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kCcBybjl3SAtIcwdWpUe3'},
      'href': 'https://api.spotify.com/v1/artists/2kCcBybjl3SAtIcwdWpUe3',
      'id': '2kCcBybjl3SAtIcwdWpUe3',
      'name': 'Lil Peep',
      'type': 'artist',
      'uri': 'spotify:artist:2kCcBybjl3SAtIcwdWpUe3'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/3aGFCoR8xGN6DKwvdzeSja'},
      'href': 'https://api.spotify.com/v1/artists/3aGFCoR8xGN6DKwvdzeSja',
      'id': '3aGFCoR8xGN6DKwvdzeSja',
      'name': 'ILOVEMAKONNEN',
      'type': 'artist',
      'uri': 'spotify:artist:3aGFCoR8xGN6DKwvdzeSja'},
     {'external_urls': {'spotify': 'https:/

In [11]:
top_2019['items'][0]['track']['id']

'2v5JTeM6hSmi5wWy7jiwrI'

## Class inheritance
Playlist is a subclass of SpotifyAPI - a few things<br>
Using the super() method alone returns a temporary object of the superclass that then allows you to call that superclass’s methods.<br>
If you're using super(), you don't need to specify self explicitly.<br>
<br>
Read this for more on Class Inheritance - https://realpython.com/python-super/

In [127]:
# Make a playlist class that inherit SpotifyAPI class methods
class Playlist(SpotifyAPI):
    tracks = set() # store a set of tracks
    artists = dict() # key:val - artist: {tracks: [{albumTitle, albumURL, imageURL, popularity, released_date},...], number of times artists appeared in playlist,}
    feature = dict() # key:value - artist
    temp_list = []
    
    def __init__(self, playlist_id, *args, **kwargs):
        super().__init__(api_key, client_id=None, client_secret=None, *args, **kwargs)
        self.playlist_id = playlist_id
        self.api_key = api_key
    
    def parse_tracks(self):
        temp_list = []
        playlist_id = self.playlist_id
        playlist_items = self.get_playlist(playlist_id)['items']
        
        if not playlist_items:
            raise Exception('No Playlist, check playlistID')
        
        for item in playlist_items:
            track_dict = parse_track(item)
            temp_list.append(track_dict)
        
        self.tracks = temp_list 
        

In [129]:
top2019 = Playlist(top_2019_id)
top2019.parse_tracks()
top2019.tracks

[{'lead': {'track': "I've Been Waiting (w/ ILoveMakonnen & Fall Out Boy)",
   'album': "I've Been Waiting (w/ ILoveMakonnen & Fall Out Boy)",
   'albumImage': 'https://i.scdn.co/image/ab67616d0000b2739067de29733d1ecf1b87e583',
   'release_date': '2019-01-31',
   'popularity': 75,
   'duration': 233}},
 {'lead': {'track': 'Hopeless Romantic (feat. Swae Lee)',
   'album': 'Hopeless Romantic (feat. Swae Lee)',
   'albumImage': 'https://i.scdn.co/image/ab67616d0000b273d7bd17179a6c41ddbddbd4bd',
   'release_date': '2018-06-21',
   'popularity': 53,
   'duration': 225}},
 {'lead': {'track': 'Moonlight',
   'album': '?',
   'albumImage': 'https://i.scdn.co/image/ab67616d0000b273806c160566580d6335d1f16c',
   'release_date': '2018-03-16',
   'popularity': 86,
   'duration': 135}},
 {'lead': {'track': 'Lover Boy',
   'album': 'Lover Boy',
   'albumImage': 'https://i.scdn.co/image/ab67616d0000b273ebefdcb9f6e825e8582bbac4',
   'release_date': '2018-03-22',
   'popularity': 67,
   'duration': 236}}

In [72]:
# List of attributes in a single item
print('Item attributes: ', top_2019['items'][0].keys())
print('Track attributes: ', top_2019['items'][0]['track'].keys())


# Interested in album, artists, duration_ms, id, popularity
print('Total Duration (seconds): ', convert_ms(top_2019['items'][0]['track']['duration_ms']))
print('Track Name: ', top_2019['items'][0]['track']['name'])


print('\nAlbum: ', top_2019['items'][0]['track']['album'].keys(), '\n')

# Get first URL of track's album image - Plan to store all images URL in a list and make a map 
print('Album Image: ', top_2019['items'][0]['track']['album']['images'][0]['url'], '\n')

print('Album Name: ', top_2019['items'][13]['track']['album']['name'])
print('Released on: ', top_2019['items'][0]['track']['album']['release_date'])

print('Album popularity: ', top_2019['items'][0]['track']['popularity'])

track_artists = [artist['name'] for artist in top_2019['items'][0]['track']['artists']]
print('Artist: ', track_artists)



Item attributes:  dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])
Track attributes:  dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
Total Duration (seconds):  233
Track Name:  I've Been Waiting (w/ ILoveMakonnen & Fall Out Boy)

Album:  dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri']) 

Album Image:  https://i.scdn.co/image/ab67616d0000b2739067de29733d1ecf1b87e583 

Album Name:  NOT ALL HEROES WEAR CAPES (Deluxe)
Released on:  2019-01-31
Album popularity:  75
Artist:  ['Lil Peep', 'ILOVEMAKONNEN', 'Fall Out Boy']


## Retrieve User Playlists
The data should be included in the url - use library urllib.
The format of the GET request is as follows - https://developer.spotify.com/console/get-playlists/?user_id=&limit=10&offset=5